In [1]:
conda init

no change     C:\Users\antho\miniconda3\Scripts\conda.exe
no change     C:\Users\antho\miniconda3\Scripts\conda-env.exe
no change     C:\Users\antho\miniconda3\Scripts\conda-script.py
no change     C:\Users\antho\miniconda3\Scripts\conda-env-script.py
no change     C:\Users\antho\miniconda3\condabin\conda.bat
no change     C:\Users\antho\miniconda3\Library\bin\conda.bat
no change     C:\Users\antho\miniconda3\condabin\_conda_activate.bat
no change     C:\Users\antho\miniconda3\condabin\rename_tmp.bat
no change     C:\Users\antho\miniconda3\condabin\conda_auto_activate.bat
no change     C:\Users\antho\miniconda3\condabin\conda_hook.bat
no change     C:\Users\antho\miniconda3\Scripts\activate.bat
no change     C:\Users\antho\miniconda3\condabin\activate.bat
no change     C:\Users\antho\miniconda3\condabin\deactivate.bat
no change     C:\Users\antho\miniconda3\Scripts\activate
no change     C:\Users\antho\miniconda3\Scripts\deactivate
no change     C:\Users\antho\miniconda3\etc\profile.d\

In [2]:
import pandas as pd
import plotly
print("Setup complete! pandas:", pd.__version__)


Setup complete! pandas: 2.3.3


In [3]:
import pandas as pd          # For data tables
import numpy as np           # For math and missing values
import plotly.express as px  # For charts
import re                    # For searching text
from pathlib import Path     # For safe file paths


In [4]:
RAW = Path("../data_raw")
CLEAN = Path("../data_clean"); CLEAN.mkdir(parents=True, exist_ok=True)
INS   = Path("../data_insights"); INS.mkdir(parents=True, exist_ok=True)

EXCEL_PATH = RAW / "Texas_insightlegi_workingcopy_nov-2025.xlsx"
print("File location:", EXCEL_PATH.resolve())


File location: C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx


In [5]:
xls = pd.ExcelFile(EXCEL_PATH)
xls.sheet_names


['tx_fee', 'tx_funds', 'tx_fund_id_desc', 'tx_additional_info', 'tx_original']

In [6]:
pen = pd.read_excel(xls, sheet_name="tx_fee")
pen.head(10)


,state,statute_id,statute_name,penalty_id,estimated_amount,penalty_type,year,penalty_amt_min,penalty_amt_max,penalty_amt_fixed,penalty_other,Notes
0,texas,tx_1,tex. penal code § 12.21,tx_1_mca1,$0 - $4000,fine for class a misdemeanor,1991,0.0,3000.0,NaN,NaN,NaN
1,texas,tx_1,tex. penal code § 12.21,tx_1_mca1,$0 - $4000,fine for class a misdemeanor,1993,0.0,4000.0,NaN,NaN,NaN
2,texas,tx_2,tex. penal code § 12.22,tx_2_mcb1,$0 - $2000,fine for class b misdemeanor,1991,0.0,1500.0,NaN,NaN,NaN
3,texas,tx_2,tex. penal code § 12.22,tx_2_mcb1,$0 - $2000,fine for class b misdemeanor,1993,0.0,2000.0,NaN,NaN,NaN
4,texas,tx_3,tex. penal code § 12.23,tx_3_mcc1,$0 - $500,fine for class c misdemeanor,1991,0.0,500.0,NaN,NaN,Fee changed from 200 to 500 in 1991
5,texas,tx_4,tex. penal code § 12.32,tx_4_fd1,$0 - $10000,first degree felony,1993,0.0,10000.0,NaN,NaN,NaN
6,texas,tx_5,tex. penal code § 12.33,tx_5_fd2,$0-$10000,second degree felony,1993,0.0,10000.0,NaN,NaN,NaN
7,texas,tx_6,tex. penal code § 12.34,tx_6_fd3,$0-$10000,third degree felony,1989,0.0,10000.0,NaN,NaN,NaN
8,texas,tx_7,tex. penal code § 12.35,tx_7_f1,$0-$10000,state jail felony,1993,0.0,10000.0,NaN,NaN,NaN
9,texas,tx_8,tex. code crim. proc. art. 102.014(d),tx_8_1,$20,court costs for parent contributing to truancy,1991,2.0,5.0,NaN,NaN,NaN


In [7]:
# 1. Clean column names (lowercase, underscores)
pen.columns = pen.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Turn text like "N/A" into proper missing values
def normalize_na(s):
    if isinstance(s, str):
        s = s.strip().lower()
        if s in {"n/a", "na", "none", ""}:
            return np.nan
    return s

for col in ["estimated_amount","penalty_other","penalty_type","statute_name"]:
    pen[col] = pen[col].apply(normalize_na)

# 3. Convert number columns to numeric
for c in ["year","penalty_amt_min","penalty_amt_max","penalty_amt_fixed"]:
    pen[c] = pd.to_numeric(pen[c], errors="coerce")

# 4. Drop rows missing key info
pen = pen.dropna(subset=["statute_id","year"]).copy()

pen.head()


,state,statute_id,statute_name,penalty_id,estimated_amount,penalty_type,year,penalty_amt_min,penalty_amt_max,penalty_amt_fixed,penalty_other,notes
0,texas,tx_1,tex. penal code § 12.21,tx_1_mca1,$0 - $4000,fine for class a misdemeanor,1991.0,0.0,3000.0,NaN,NaN,NaN
1,texas,tx_1,tex. penal code § 12.21,tx_1_mca1,$0 - $4000,fine for class a misdemeanor,1993.0,0.0,4000.0,NaN,NaN,NaN
2,texas,tx_2,tex. penal code § 12.22,tx_2_mcb1,$0 - $2000,fine for class b misdemeanor,1991.0,0.0,1500.0,NaN,NaN,NaN
3,texas,tx_2,tex. penal code § 12.22,tx_2_mcb1,$0 - $2000,fine for class b misdemeanor,1993.0,0.0,2000.0,NaN,NaN,NaN
4,texas,tx_3,tex. penal code § 12.23,tx_3_mcc1,$0 - $500,fine for class c misdemeanor,1991.0,0.0,500.0,NaN,NaN,Fee changed from 200 to 500 in 1991


In [8]:
#New Statutes
import pandas as pd
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx")
df = pd.read_excel(file_path, sheet_name="tx_fee")

# --- Clean year column ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.dropna(subset=["year"])

# --- Aggregate: count statutes per year ---
statutes_per_year = (
    df.groupby("year")
    .size()
    .reset_index(name="statute_count")
)

# --- Save aggregated data (optional) ---
statutes_per_year.to_csv("new_statutes_per_year_counts.csv", index=False)

# --- Plot ---
plt.figure(figsize=(8,5))
plt.bar(statutes_per_year["year"], statutes_per_year["statute_count"])
plt.title("New Statutes Introduced per Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\TX_visuals")
save_folder.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist
save_path = save_folder / "01_new_statutes.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()

In [9]:
# STATUTES CHANGED PER YEAR
import os
os.environ.pop("MPLBACKEND", None)  # Fix Matplotlib backend issue before importing
import matplotlib
matplotlib.use("TkAgg")

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx")
df = pd.read_excel(file_path, sheet_name="tx_fee")

# --- Clean data ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["statute_id"] = df["statute_id"].astype(str)

# --- Filter out repealed rows and missing years ---
mask_repealed = df["penalty_type"].astype(str).str.contains("repeal", case=False, na=False)
df_filtered = df.loc[~mask_repealed & df["year"].notna()]

# --- Count unique statutes that had changes per year ---
statute_changes_per_year = (
    df_filtered.groupby("year")["statute_id"]
    .nunique()
    .reset_index()
    .rename(columns={"statute_id": "unique_statutes_changed"})
)

# --- Save output CSV ---
output_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_output\statutes_changed_per_year_cleantx.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
statute_changes_per_year.to_csv(output_path, index=False)

# --- Plot ---
plt.figure(figsize=(8,5))
plt.bar(statute_changes_per_year["year"], statute_changes_per_year["unique_statutes_changed"])
plt.title("Number of Statutes with Changes per Year (Excluding Repealed)")
plt.xlabel("Year")
plt.ylabel("Number of Unique Statutes")
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\TX_visuals")
save_folder.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist
save_path = save_folder / "02_statutes_changed.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()


In [10]:
# 10 PENALTIES OVER TIME GRAPH
import os
os.environ.pop("MPLBACKEND", None)
import matplotlib
matplotlib.use("TkAgg")

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import textwrap

# --- Load data ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx")
df = pd.read_excel(file_path, sheet_name="tx_fee")

# --- Clean columns ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["penalty_type"] = df["penalty_type"].astype(str)
df["statute_id"] = df["statute_id"].astype(str)

# --- Filter to valid years ---
df = df.loc[df["year"].notna()]

# --- Identify top 10 most frequent penalty types overall ---
top_penalty_types = df["penalty_type"].value_counts().head(10).index
df_top = df[df["penalty_type"].isin(top_penalty_types)]

# --- Pivot: unique statutes per year per penalty type ---
pivot = (
    df_top.pivot_table(
        index="year",
        columns="penalty_type",
        values="statute_id",
        aggfunc="nunique"
    )
    .fillna(0)
)

# --- Wrap long legend labels ---
wrapped_cols = ['\n'.join(textwrap.wrap(col, 30)) for col in pivot.columns]

# --- Clear any auto-figures ---
plt.close('all')

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 6))
pivot.plot(kind="line", linewidth=2, ax=ax)

ax.set_title("Top 10 Penalty Types Over Time")
ax.set_xlabel("Year")
ax.set_ylabel("Unique Statutes (Non-Repealed)")

# Use wrapped labels in the legend
ax.legend(
    wrapped_cols,
    title="Penalty Type",
    bbox_to_anchor=(1.05, 1),
    loc="upper left"
)

ax.grid(True, linestyle="--", alpha=0.6)

# Adjust margins to fit legend
plt.subplots_adjust(left=0.1, right=0.8)
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\TX_visuals")
save_folder.mkdir(parents=True, exist_ok=True)
save_path = save_folder / "03_10_penalties.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()



In [11]:
# AVERAGE PENALTY OVER TIME GRAPH
import os
os.environ.pop("MPLBACKEND", None)  
import matplotlib
matplotlib.use("TkAgg") 

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx")
df = pd.read_excel(file_path, sheet_name="tx_fee")

# --- Clean numeric columns ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
for col in ["penalty_amt_min", "penalty_amt_max", "penalty_amt_fixed", "estimated_amount"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Compute effective penalty (take whichever is non-null and highest) ---
df["penalty_effective"] = df[["penalty_amt_fixed", "penalty_amt_max", "estimated_amount"]].max(axis=1)

# --- Filter for valid years and positive values ---
df_clean = df[(df["year"].notna()) & (df["penalty_effective"] > 0)]

# --- Average per year ---
avg_penalty = (
    df_clean.groupby("year")["penalty_effective"]
    .mean()
    .reset_index()
    .rename(columns={"penalty_effective": "avg_penalty"})
)

# --- Plot ---
plt.figure(figsize=(9, 5))
plt.plot(avg_penalty["year"], avg_penalty["avg_penalty"], marker="o", linewidth=2)
plt.title("Average Penalty Amount Over Time")
plt.xlabel("Year")
plt.ylabel("Average Penalty (USD)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\TX_visuals")
save_folder.mkdir(parents=True, exist_ok=True)
save_path = save_folder / "04_average_penalties.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()


In [12]:
def winsorize(series, lower=0.025, upper=0.975):
    return series.clip(lower=series.quantile(lower), upper=series.quantile(upper))

df["penalty_winsorized"] = winsorize(df["penalty_effective"])


In [13]:
import pandas as pd
import re

# Load your Excel file
pen = pd.read_excel(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\Texas_insightlegi_workingcopy_nov-2025.xlsx")

# Define discretionary patterns
DISCRETION_PATTERNS = [
    r"judge", r"court", r"discretion", r"reasonable",
    r"to be determined", r"as determined", r"at (the )?court'?s discretion", r"by the court"
]
discretion_re = re.compile("|".join(DISCRETION_PATTERNS), flags=re.IGNORECASE)

# Function to check if text is discretionary
def is_discretionary(text):
    if not isinstance(text, str):
        return False
    return bool(discretion_re.search(text))

pen["is_discretionary_text"] = pen["penalty_other"].apply(is_discretionary)

# Mark numeric amounts
pen["has_min"]   = pen["penalty_amt_min"].notna() & (pen["penalty_amt_min"] > 0)
pen["has_max"]   = pen["penalty_amt_max"].notna() & (pen["penalty_amt_max"] > 0)
pen["has_fixed"] = pen["penalty_amt_fixed"].notna() & (pen["penalty_amt_fixed"] > 0)
pen["has_numeric_amount"] = pen[["has_min","has_max","has_fixed"]].any(axis=1)

# Classify amount type
def classify_amount_type(row):
    if row["has_fixed"]:
        return "fixed"
    if row["has_min"] or row["has_max"]:
        return "range"
    if row["is_discretionary_text"]:
        return "discretionary_text"
    return "none"

pen["amount_type"] = pen.apply(classify_amount_type, axis=1)
print(pen[["penalty_id","year","penalty_other","amount_type"]].head(10))

  penalty_id  year penalty_other amount_type
0  tx_1_mca1  1991           NaN       range
1  tx_1_mca1  1993           NaN       range
2  tx_2_mcb1  1991           NaN       range
3  tx_2_mcb1  1993           NaN       range
4  tx_3_mcc1  1991           NaN       range
5   tx_4_fd1  1993           NaN       range
6   tx_5_fd2  1993           NaN       range
7   tx_6_fd3  1989           NaN       range
8    tx_7_f1  1993           NaN       range
9     tx_8_1  1991           NaN       range


In [14]:
pen.to_csv(CLEAN / "penalties_TX.csv", index=False)
print("Cleaned file saved to data_clean/")


Cleaned file saved to data_clean/


In [15]:
pen["year"] = pd.to_numeric(pen["year"], errors="coerce").astype("Int64")


In [16]:
# Ensure year is numeric
pen["year"] = pd.to_numeric(pen["year"], errors="coerce").astype("Int64")

# Averages of penalty amounts per year
trend_amt = (
    pen.groupby("year", as_index=False)
       .agg(
           avg_min=("penalty_amt_min","mean"),
           avg_max=("penalty_amt_max","mean"),
           avg_fixed=("penalty_amt_fixed","mean"),
           n_statutes=("statute_id","nunique")
       )
       .sort_values("year")
)

# Share of each amount type per year
type_counts = (
    pen.groupby(["year","amount_type"])["penalty_id"]
       .count()
       .reset_index(name="count")
)

year_totals = type_counts.groupby("year")["count"].sum().rename("year_total")
type_share = type_counts.merge(year_totals, on="year")
type_share["pct"] = (100 * type_share["count"] / type_share["year_total"]).round(1)

trend_amt.to_csv(INS / "penalty_trend_TX.csv", index=False)
type_share.to_csv(INS / "penalty_amount_type_share_TX.csv", index=False)

print("Trend and share tables saved.")



Trend and share tables saved.


In [17]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import sys
import subprocess

# --- Ensure Kaleido is installed ---
try:
    import kaleido
except ImportError:
    print("Kaleido not found. Installing now...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "kaleido"])
    import kaleido

# --- Example data (replace with your own DataFrames) ---
# trend_amt = ...
# type_share = ...

# --- Create Line chart of average penalties ---
cols = ["avg_min","avg_max","avg_fixed"]
fig1 = px.line(trend_amt, x="year", y=cols, markers=True,
               title="Texas – Average Penalty Levels Over Time")

# --- Create Area chart of amount types ---
comp_pivot = type_share.pivot(index="year", columns="amount_type", values="pct").fillna(0).reset_index()
fig2 = px.area(comp_pivot, x="year",
               y=[c for c in comp_pivot.columns if c != "year"],
               title="Texas – Share of Amount Types by Year",
               groupnorm="fraction")
fig2.update_yaxes(ticksuffix="%")

# --- Output directory ---
output_dir = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\TX_visuals")
output_dir.mkdir(parents=True, exist_ok=True)

# --- Save figures ---
fig1.write_image(output_dir / "05_line_chart.png")
fig2.write_image(output_dir / "06_area_chart.png")

# --- Show figures in notebook / script ---
fig1.show()
fig2.show()




In [18]:
missing = (100 * pen[["penalty_amt_min","penalty_amt_max","penalty_amt_fixed"]].isna().mean()).round(1)
print("Missing numeric fields (%):\n", missing)
print("\nAmount types overall:\n", pen["amount_type"].value_counts(dropna=False))


Missing numeric fields (%):
 penalty_amt_min      65.5
penalty_amt_max      65.5
penalty_amt_fixed    57.3
dtype: float64

Amount types overall:
 amount_type
fixed    47
range    38
none     25
Name: count, dtype: int64
